## UNet의 이해
### U-Net 
- Biomedical 분야에서 **이미지 분할**을 목적으로 제안된 **End-to-End** 방식의 **Fully-Convolutional Network** 기반 모델

#### 이미지 분할(image segmentation)
- 이미지의 모든 픽셀이 어떤 카테고리에 속하는지 분류하는 것
  - 이미지 분류(image classification)과는 달리 픽셀단위의 분류를 수행하므로 좀 더 어려움

#### End-to-End : 끝과 끝만 보고 중간의 과정은 기계가 알아서 학습
- 어떤 문제를 해결해 나가는데 있어서 필요한 스텝들을 하나의 신경망을 통해서 재배치 하는과정
- 이는 딥러닝의 중요한 특징으로, 딥러닝이 hand-crafted feature을 필요로 하는 머신러닝보다 더 하위 개념임을 알 수 있다
  - 크게는 두 개로 나누어서 하나의 목적에 부합하는 신경망 네트워크를 구축하고 이어서 다음 단계의 목적에 부합하는 네트워크를 생성하는 식
    - 예를들어 얼굴인식 기술을 만들 때 **사진에서 사람의 얼굴 부분만을 인식하는 부분 & 그 추출한 이미지 부분이 모델이 보유하고 있는 사람들의 얼굴값과 일치하는지 판별하는 부분** 두 가지로 나누어 학습
    
#### Fully-Convolutional Network (FCN)   
- 이미지 분류와 분할은 세 과정이다
  - Convolutionalization > Deconvolution(Upsampling) > skip architecture
  - 이미지 분류 모델은 기본적으로 fc(fully connected) layer로 구성
  - 근데 이러면 위치정보가 사라지고 입력이미지 크기가 고정되는 문제 발생
  - 이미지 분할은 픽셀에 대해 클래스를 구분하고 인스턴스 및 배경을 분할하는 것이기 때문에 위 두 문제점을 해결해야했음
- 이러한 한계를 보안하기위해 fc-layer를 Conv-layer로 대체
  - 그랬더니 Feature map이 너무 coarse(거친, 알맹이가 큰) 함
  - 그래서 Coarase map을 원본 이미지 크기에 가까운 Dense map으로 변환할 필요가 있음
    - 그 방법으로는 **Interpolation**, **Deconvolution**, Unpooling, Shift and stitch 의 방법이 있음
    - 풀링을 안하면(unpooling) 파라미터 수가 급격히 증가하고 학습시간이 많이 걸림
    - 그래서 Upsamplig 방법 고려

#### Bilinear Interpolation
- 이미지 확대하는 대표적인 방법

#### Backwards convolution (=Upsampling = Deconvolution)
- Stride가 2 이상인 Convolution 연산의 경우 이미지에 대해 크기가 줄어든 특징맵을 출력하는 것을 Down-sampling이라 함
- 그 반대가 Up-sampling
  - 즉 input보다 큰 filter로 output 추출
  - 이때 사용하는 Filter의 가중치 값이 학습 파라미터
- 근데 Upsamling해도 feature map의 크기가 너무 작아서 dense map이 여전히 거침  => Skip Architecture

#### Skip Architecture
- Deep & Coarse 레이어의 의미적(Semantic) 정보와 Shallow&fine 층의 외관적 정보를 결합한 skip architecture 정의
![1_pbEMiRkXz65ejneuuToK3g.webp](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*pbEMiRkXz65ejneuuToK3g.png)

- U-Net은 이미지의 전반적인 컨텍스트 정보를 얻기 위한 네트워크와 정확한 지역화를 위한 네트워크가 대칭으로 구성되어 있다
![1*qNdglJ1ORP3Gq77MmBLhHQ.png](https://miro.medium.com/v2/resize:fit:828/format:webp/1*qNdglJ1ORP3Gq77MmBLhHQ.png)

![1*8T3P2kenhju8dNXgRPVyfg.png](https://miro.medium.com/v2/resize:fit:828/format:webp/1*8T3P2kenhju8dNXgRPVyfg.png)

**Contracting Path** : 입력이미지의 context 포착을 목적으로 구성
- 3x3 convolutions을 두차례씩 반복 (padding=0)
- 활성화 함수는 ReLU
- 2x2 max-pooling (stride : 2)
- Down-sampling 마다 채널의 수를 2배로 늘림

![1*gLytTsUQ52fsxDRdUbONuw.png](https://miro.medium.com/v2/resize:fit:828/format:webp/1*gLytTsUQ52fsxDRdUbONuw.png)

![1*Iyb7ALgrB4zhZaWhG4M7ZQ.png](https://miro.medium.com/v2/resize:fit:828/format:webp/1*Iyb7ALgrB4zhZaWhG4M7ZQ.png)

**Expanding Path** : 세밀한 Localization을 위한 구성. 높은차원의 채널을 갖는 Up-sampling. 얕은 레이어의 특징맵을 결합
- contracting path와 반대연산
- 2x2 convolution("up-convolution")
- 3x3 convolutions을 두 차례씩 반복
- up-Conv를 통한 up-sampling마다 채널의 수를 반으로 줄임
- 활성화함수는 ReLU
- Up-Conv된 특징맵은 Contracting path의 테두리가 Cropped된 특징맵과 concatenation함
- 마지막 레이어에 1x1 convolution 연산

#### Overlap-Tite Input
- 크기가 큰 이미지의 경우 이미지 전체를 사용하는 대신 overlap-title 전략 사용
- 이미지를 타일로 나눠서 입력으로 사용 => 다음 타일에 대한 분할을 얻기 위해서는 이전 입력의 일부분이 포함되어야함
- 경계 부분 픽셀에 대한 분할을위해 0의 패딩 대신 경계 부분의 미러링을 이용한 Extrapolation 기법 사용
![1*wbHb3A9-FeaE4xvKwrswEg.png](https://miro.medium.com/v2/resize:fit:828/format:webp/1*wbHb3A9-FeaE4xvKwrswEg.png)

#### Touching cells separation
- 세포 사이의 경계를 포착하기 위해 Ground-Truth에 대한 Weight map을 구해 학습 반영
![1*hE1UHMq_sX-jkQF36c3RBw.png](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*hE1UHMq_sX-jkQF36c3RBw.png)
- $w_{c}:\Omega \to R$ : Weight map to balance the class frequencies
- $d_{1}:\Omega \to R$ : The distance to the border of the nearest cell
- $d_{2}:\Omega \to R$ : The distance to the border of the second nearest cell

#### Training
- 출력값은 픽셀 단위의 Softmax로 예측
![1*l-rSSk-VEUszzaKLZyTphA.png](https://miro.medium.com/v2/resize:fit:828/format:webp/1*l-rSSk-VEUszzaKLZyTphA.png)
  - a(x)는 픽셀 x의 활성도, K는 클래스 수

- 따라서 손실함수는 Cross Entropy 함수가 사용. 단 Touching cell 분리 고려를 위해 가중치 맵 손실이 포함됨
![1*kWKvXdjllmzYzV0mEIBj0A.png](https://miro.medium.com/v2/resize:fit:828/format:webp/1*kWKvXdjllmzYzV0mEIBj0A.png)
  - momentum : 0.99